In [19]:
import requests
from bs4 import BeautifulSoup
import re
import threading
from datetime import datetime
import pandas as pd
import time
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
import concurrent.futures
from urllib.parse import urlparse, parse_qs

In [20]:
data = {
    "keywords": "anies",
    "since_time": "2023-09-01",
    "until_time": "2023-11-10"
}

In [21]:
# input_date_str = datetime.strftime(data['since_time'], "%Y-%m-%d")
input_date_str=data['since_time']
print(input_date_str)
# headers = {
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
#     }
# url = f"https://index.okezone.com/bydate/channel/2023/09/01/1"
# response = requests.get(url, headers=headers)
# soup = BeautifulSoup(response.text, 'html.parser')
# response = requests.get(url, headers=headers)
# soup = BeautifulSoup(response.text, 'html.parser')
# articles = soup.find_all('h4', {"class": "f17"})
# for article in articles:
#     link = article.find('a')['href']
#     print(link)

2023-09-01


In [22]:
headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
url = f"https://index.okezone.com/bydate/channel/2023/09/01/1"
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
# print(soup)
cek = soup.find('div', {"class": "time r1 fl bg1 b1"})
links=cek.find_all('a')
all_link=[]
for link in links:
        if link and 'href' in link.attrs:
            link_href = link['href']
            all_link.append(link_href)
link=all_link[-1]
# print(link)
parsed_url = urlparse(link)
# print(parsed_url)
number = parsed_url.path.split("/")[-2]
# print(number)

In [23]:
def scrape_links(date, page_number):
    input_date_str = datetime.strftime(date, "%Y-%m-%d")
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    url = f"https://index.sindonews.com/index/5/{page_number}?t={input_date_str}"
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    articles = soup.find_all('div', {"class": "indeks-title"})
    
    links = []
    for article in articles:
        link = article.find('a')['href']
        links.append(link)
        
    print(f"Scraped {len(links)} links from page {page_number} url {url}")
    return links

In [24]:

def scrape_link_per_day(date, max_threads=5):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    current_date = datetime.strptime(date, "%Y-%m-%d")
    url = f"https://index.sindonews.com/index/5/0?t={current_date}"
    response = requests.get(url + "1", headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    cek = soup.find('div', {"class": "pagination"})
    links = cek.find_all('a')
    all_link = [link['href'] for link in links if link and 'href' in link.attrs]

    if not all_link:
        return []

    last_link = all_link[-1]

    # Extract the "page" parameter value from the last link
    parsed_url = urlparse(last_link)
    page_value = parsed_url.path.split("/")[-2]
    print(page_value)
    # Check if page_value is an integer
    try:
        page_number = int(page_value)
    except (TypeError, ValueError):
        page_number = 0

    page_links = []

    with concurrent.futures.ThreadPoolExecutor(max_threads) as executor:
        # Use list comprehension to submit tasks to the thread pool
        futures = [executor.submit(scrape_links, current_date, index) for index in range(0, page_number + 20, 20)]

        # Collect results from completed futures
        for future in concurrent.futures.as_completed(futures):
            page_links.extend(future.result())

    return page_links

In [25]:
scrape_link_per_day('2023-11-02')

0
Scraped 20 links from page 0 url https://index.sindonews.com/index/5/0?t=2023-11-02


['https://nasional.sindonews.com/read/1241963/14/persiapan-operasi-lilin-2023-korlantas-polri-cek-jalur-tol-semarang-surabaya-1698941504',
 'https://nasional.sindonews.com/read/1241921/13/hakim-wahiduddin-adams-diperiksa-secara-khusus-oleh-mkmk-ini-alasannya-1698937888',
 'https://nasional.sindonews.com/read/1241919/94/dukung-petani-berbasis-ebt-kilang-pertamina-internasional-resmikan-program-desa-energi-berdikari-kalijaran-1698937901',
 'https://nasional.sindonews.com/read/1241909/12/besok-ziarah-ke-makam-bung-karno-di-blitar-mahfud-md-tak-ada-persiapan-khusus-1698937911',
 'https://nasional.sindonews.com/read/1241897/12/ganjar-mahfud-bersama-tpn-bahas-isu-strategis-persiapkan-debat-pilpres-2024-1698934301',
 'https://nasional.sindonews.com/read/1241889/15/muhammadiyah-salurkan-bantuan-rp13-miliar-untuk-palestina-1698934274',
 'https://nasional.sindonews.com/read/1241863/12/mahfud-susul-ganjar-cek-kesiapan-debat-capres-cawapres-1698934281',
 'https://nasional.sindonews.com/read/124185